## implementation of Bayesian Personalized Ranking for Implicit Feedback

https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf  
  
Neighborhood methods without K nearest  
needs to be modified

In [1]:
import numpy as np
import data
import random
from timeit import default_timer as timer

In [2]:
ml_100k = data.ml_100k
train = data.train
test = data.test

In [3]:
class MatrixFactorization():
    
    def __init__(self, data, train, test, learning_rate, reg_param):
        """
        param data : Rating Matrix
        param train : Rating Matrix for train
        param test : Rating Matrix for test
        param learning_rate : alpha on weight update
        param reg_param : regularization parameter
        """
        
        self._A_I = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(data), dtype = np.float64)
        self._X = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(train), dtype = np.float64) # create X matrix : implicit feedbacks (binary)
        self._X_test = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(test), dtype = np.float64)
        self._num_users, self._num_items = train.shape
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        
        
    def fit(self):
        """
        training Matrix Factorization : update matrix latent weight and bias
        """
        
        self._C = np.random.normal(0, scale = 1.0/self._num_items, size=(self._num_items, self._num_items))
        
        # train until cost converges
        count = 0
        self._training_process = []
        for j in range(10):
            start = timer()
            for i in range(10000) :    

                count += 1
                # randomly choice _ Bootstrap
                u = random.choice(self._X.nonzero()[0])
                i = random.choice(self._X[u].nonzero()[0]) 
                j = random.choice(np.argwhere(self._X[u] == 0).T[0]) 
                self.gradient_descent(u, i, j)
            print("complete 10000 iterations, time :%.4f" % (timer()-start))

            start_AUC = timer()
            AUC = self.compute_AUC()
            self._training_process.append((count, AUC))
            print("Iteration : %d, AUC = %.4f, AUC computation time: %.4f" % (count, AUC, timer()-start_AUC))
                
    
    
    def sigmoid(self, x):
        """
        return sigmoid 
        """
        return 1 / (1 + np.exp(-x))

    
    
    def gradient_descent(self, u, i, j):
        """
        gradient descent function
        param u : user index
        param i : item index i
        param j : item index j
        """
        # compute Xuij
        xui_hat = np.sum(self._X[u]*self._C[i]) - self._C[i, i]
        xuj_hat = np.sum(self._X[u]*self._C[j]) - self._C[j, j]
        xuij_hat = xui_hat - xuj_hat
        
        # l array for update (user updated)
        l = self._X[u].nonzero()[0]
        l_array = np.setdiff1d(l, i)
        
        sigmoid = self.sigmoid(xuij_hat) * np.exp(-xuij_hat)
        self._C[i, l_array] += self._learning_rate * (sigmoid - self._reg_param * self._C[i, l_array])
        self._C[l_array, i] += self._learning_rate * (sigmoid - self._reg_param * self._C[l_array, i])
        self._C[j, l_array] += self._learning_rate * (-1 * sigmoid - self._reg_param * self._C[j, l_array])
        self._C[l_array, j] += self._learning_rate * (-1 * sigmoid - self._reg_param * self._C[l_array, j])
    
    
        
    def compute_AUC(self):
        """
        compute AUC measure
        """
        # make prediction matrix
        self._X_hat = np.zeros((self._num_users, self._num_items))
        for u in range(self._num_users):
            for i in range(self._num_items):
                self._X_hat[u, i] = np.sum( self._X[u] * self._C[i] ) - self._C[i, i]
        
        u_nonzero, i_nonzero = self._X_test.nonzero()
        num = 0
        
        for u in u_nonzero :
            temp = 0
            # non-zeros in test set
            temp_i = self._X_test[u].nonzero()[0]
            # zeros in whole data set
            temp_j = np.argwhere(self._A_I[u] == 0).T[0]
            for i in temp_i :
                for j in temp_j :
                    if self._X_hat[u, i] > self._X_hat[u, j] :
                        temp += 1
            num += (temp / (len(temp_i)*len(temp_j)))
        auc = num / len(u_nonzero)
        
        return auc

In [4]:
np.random.seed(7)
np.seterr(all="warn")

factorizer = MatrixFactorization(ml_100k, train, test, learning_rate=0.1, reg_param=0.01)

# regression parameter 2개
factorizer.fit()

complete 10000 iterations, time :103.0416
Iteration : 10000, AUC = 0.8174, AUC computation time: 1437.7876
complete 10000 iterations, time :104.0562
Iteration : 20000, AUC = 0.8354, AUC computation time: 1447.9958
complete 10000 iterations, time :100.4978
Iteration : 30000, AUC = 0.8385, AUC computation time: 1478.5918
complete 10000 iterations, time :100.6912
Iteration : 40000, AUC = 0.8405, AUC computation time: 1426.7284
complete 10000 iterations, time :100.6411
Iteration : 50000, AUC = 0.8471, AUC computation time: 1484.2030
complete 10000 iterations, time :101.5241
Iteration : 60000, AUC = 0.8457, AUC computation time: 1442.6704
complete 10000 iterations, time :101.4006
Iteration : 70000, AUC = 0.8499, AUC computation time: 1487.9572
complete 10000 iterations, time :100.2331
Iteration : 80000, AUC = 0.8424, AUC computation time: 1471.7432
complete 10000 iterations, time :100.3637
Iteration : 90000, AUC = 0.8443, AUC computation time: 1461.4148
complete 10000 iterations, time :100.